Загружаем необходимые библиотеки

In [1]:
import numpy as np
import pandas as pd
import nltk

In [2]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import Normalizer
from sklearn.svm import LinearSVC
from sklearn.cross_validation import cross_val_score#, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from scipy import interp

from nltk.corpus import stopwords


C:\Users\Alexal\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Считываем данные из текстов в той же папке, что и ноутбук

In [3]:
train_pd =pd.read_csv('products_sentiment_train.tsv', sep='\t', header = -1)

In [4]:
train_pd = train_pd.rename(index=str, columns={0: "text", 1: "label"})

In [5]:
train_pd.head()

,text,label
0,"2 . take around 10,000 640x480 pictures .",1
1,i downloaded a trial version of computer assoc...,1
2,the wrt54g plus the hga7t is a perfect solutio...,1
3,i dont especially like how music files are uns...,0
4,i was using the cheapie pail ... and it worked...,1


In [6]:
test_pd =pd.read_csv('products_sentiment_test.tsv', sep='\t', header = 0, index_col="Id")

In [7]:
test_pd.head()

,text
Id,
0,"so , why the small digital elph , rather than ..."
1,3/4 way through the first disk we played on it...
2,better for the zen micro is outlook compatibil...
3,6 . play gameboy color games on it with goboy .
4,"likewise , i 've heard norton 2004 professiona..."


In [8]:
print len(train_pd['text'])

2000


In [9]:
texts = np.array(train_pd['text'])
labels = np.array(train_pd['label'])
test_texts = np.array(test_pd['text'])

In [10]:
texts.shape

(2000L,)

### Первая попытка

Я сдавал это задание дважды, по расписанию + пересдача после завершения 7 недели. Ниже представлен первый код, который дал мне оценку 0.815000, но не позволил войти в Top50 и получить максимальный бал.

In [11]:
tf_vectorizer = CountVectorizer()
tf = tf_vectorizer.fit_transform(texts)

In [12]:
tf

<2000x3973 sparse matrix of type '<type 'numpy.int64'>'
	with 29639 stored elements in Compressed Sparse Row format>

In [13]:
tf_vectorizer.get_params

<bound method CountVectorizer.get_params of CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)>

In [14]:
#def text_classifier(vectorizer, transformer, classifier):
def text_classifier(vectorizer, classifier):
    return Pipeline(
            [("vectorizer", vectorizer),
 #           ("transformer", transformer),
            ("classifier", classifier)]
        )

In [15]:
nltkstop = nltk.corpus.stopwords.words('english')

Выбираем модель 

In [16]:
for clf in [LogisticRegression]:
    print clf
    print cross_val_score(text_classifier(CountVectorizer(ngram_range=(1,2), stop_words = nltkstop), clf()), texts, labels, cv=5).mean()
    print "\n"

<class 'sklearn.linear_model.logistic.LogisticRegression'>
0.748498134363




In [17]:
for clf in [LogisticRegression]:
    print clf
    print cross_val_score(text_classifier(CountVectorizer(ngram_range=(1,2)), clf()), texts, labels, cv=5).mean()
    print "\n"

<class 'sklearn.linear_model.logistic.LogisticRegression'>
0.770504443778




Получаем предсказания ответов

In [18]:
random_state=2
clf_pipeline = Pipeline(
            [("vectorizer", CountVectorizer(ngram_range=(1,2))),
            ("classifier", LogisticRegression())]
        )


clf_pipeline.fit(texts, labels)

print clf_pipeline

Pipeline(steps=[('vectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
    ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])


In [19]:
print clf_pipeline.predict(test_texts)

[1 0 1 1 0 0 1 0 0 1 1 0 1 0 1 1 1 0 1 1 1 1 0 1 1 0 1 0 1 1 1 0 1 1 1 1 1
 1 1 0 0 1 1 1 1 0 1 1 1 0 1 0 1 1 0 1 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 0 1 0
 1 0 0 0 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1 1 0 1 1 0 1 1 1 1 1
 1 0 1 1 1 1 1 1 0 0 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 0 1 0 1 1 0 1 0 1 1 0 0
 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 0 1 1 1 0 1 1 0 1 0 1 0 1 1
 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 0 1 1 1 0 1 1
 1 1 1 1 0 0 1 0 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 0
 1 1 0 0 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 0 1 1 0
 1 1 1 1 0 1 1 1 0 1 0 1 0 1 1 1 1 1 1 0 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0
 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 0 1 0 1 0 1 1 1 1 0 1 1 0 1 1 0 1 1 1 1 0
 1 1 1 1 1 0 0 0 0 1 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1
 0 1 0 1 1 1 0 0 0 1 0 0 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 1
 1 1 0 1 1 1 1 1 1 1 0 1 

In [20]:
test_pd['y'] = clf_pipeline.predict(test_texts)
test_pd.drop('text', axis=1).to_csv("predictions.csv")

Попытка 2 со стоп-словами

In [21]:
random_state=2
clf_pipeline = Pipeline(
            [("vectorizer", CountVectorizer(ngram_range=(1,2), stop_words = nltkstop)),
            ("classifier", LogisticRegression())]
        )


clf_pipeline.fit(texts, labels)

print clf_pipeline

Pipeline(steps=[('vectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None,
        stop_words=[u...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])


In [22]:
print clf_pipeline.predict(test_texts)

[1 0 1 1 0 0 1 0 0 1 1 0 1 1 1 1 0 0 1 1 1 1 0 1 1 0 1 0 1 1 1 0 1 1 1 1 1
 1 1 0 0 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 0 0 1 1 1 1 0 0 0 1 0 1 1 0 1 1 1
 1 0 1 0 1 1 1 0 0 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 0
 1 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 0 1 1 0 0
 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 1 0 0 1 1 1 1 0 1 1 1 0 1 1 0 1 1 1 0 1 1
 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1 0
 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 1 1 1 1 1 1 1 0
 1 1 1 1 1 1 1 1 1 0 0 1 0 1 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 0 1 0 0 0 1 1 1 1 0 0 1 0 1 0 0 1 1 1 1 1
 1 1 1 0 1 1 0 0 0 1 0 0 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1
 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1 1 1 1 0 0 1 1 1 0 0 1
 1 0 0 1 1 1 1 1 1 1 0 0 

In [23]:
test_pd['y'] = clf_pipeline.predict(test_texts)
test_pd.drop('text', axis=1).to_csv("predictions.csv")

Кегль подсказывает, что первый вариант был сильнее

### Пересдача

Отюда начинается код пересдачи, который, неожиданно для меня самого, позволил попасть в Top7!  Я не сохранил все итерации, т.к. правил папйплайн "по живому", но путь выглядел следующим образом:

1. По совету на Slack я поменял векторизатор и классификатор на Tfidf и LinearSVC
2. ngram_range оставил (1, 2), так как он помог улучшить качество во время предыдущей попытки
3. Основного прироста удалось добиться за счет балансировки классов для классификатора (class_weight='balanced') и использование нормализатора (добавлен в пайплайн). В подборе этих параметров помог гугл и тематические лекции предыдущих курсов.


In [24]:
pipeline = Pipeline([
    ('vect', TfidfVectorizer(ngram_range=(1, 2))),
    ('norm', Normalizer()),
    ('clf', LinearSVC(class_weight='balanced'))
])

In [25]:
pipeline.fit(train_pd.text, train_pd.label)

Pipeline(steps=[('vect', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm=u'l2', preprocessor=None, smooth_idf=True...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [26]:
test_pd['y'] = pipeline.predict(test_pd.text)
test_pd.drop('text', axis=1).to_csv("predictions.csv")

In [27]:
test_pd.head(5)

,text,y
Id,,
0,"so , why the small digital elph , rather than ...",1
1,3/4 way through the first disk we played on it...,0
2,better for the zen micro is outlook compatibil...,1
3,6 . play gameboy color games on it with goboy .,1
4,"likewise , i 've heard norton 2004 professiona...",0


В целом, я считаю, что мне повезло натолкнуться на параметры, которые разом подняли мой рейтинг на несколько десятков строк. Но для меня остается загадкой, что делали люди, набравшие 98% точности..